In [111]:
!pip install pandas
!pip install mdextractor
!pip install --upgrade langchain
!pip install --upgrade python-dotenv
!pip install --upgrade langchain_community


In [113]:
import os
from langchain.llms import OpenAI 
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from mdextractor import extract_md_blocks



## Reading the csv and generating a list of dictionaries

In [114]:
df = pd.read_csv('data.csv')


In [115]:
dataList=[]
column_names = df.columns
for r in df.iterrows():
    dct={}
    for i in range(len(column_names)):
        dct[column_names[i]]=r[1].iloc[i]
    dataList.append(dct)

    

## Langchain

In [116]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.4)

In [117]:
template = """  
### INFORMATION
Name : {name}
Field Of Study : {study_field}
Year of Study : {year}
List of Subjects : {subjects}
Preferred Learning Style : {learning_style}
Personal Objectives: {objectives}
Challenges: {challenges}
Extracurricular Activities : {ECA}

Carefully consider the data provided and reason through the provided instructions. 
1.) Identify the students goals 
2.) Analyze the students challenges and learning styles to choose activities that suit them 
3.) Make sure to leave buffer time in between. 
4.) Try to incorporate extra-curricular into study sessions. 
5.) Create a timetable and mark it such, ensure that you give the timetable in markdown, and add a description for each entry
6.) Your output should be descriptive and not too concise
7.) Ensure that the activities are actionable things.


"""
# create a prompt template using above-defined template string 
prompt_template = PromptTemplate.from_template(
    template=template
)
data=dataList[10]
final_prompt = prompt_template.format(
    name = {data["Name"]},
    study_field= {data["Field of study"]},
    year = {data['Year of study']},
    subjects = {data['List of subjects']},
    learning_style = {data['Preferred Learning Styles']},
    objectives = {data['Personal Objectives']},
    challenges = {data['Challenges']},
    ECA = {data['Extracurricular activities']},
)
result = chat.invoke(final_prompt).content


In [130]:
from IPython.display import Markdown, display
blocks = extract_md_blocks(result)
blocks =  "```markdown\n" + blocks[0] + "\n```"

display(Markdown(blocks))


```markdown
| Time          | Activity                                          | Description                                                                                      |
|---------------|---------------------------------------------------|--------------------------------------------------------------------------------------------------|
| 8:00 AM - 9:00 AM | **Quantum Mechanics Study Session**            | Focus on foundational concepts in quantum mechanics. Use online resources and videos for engaging content. |
| 9:00 AM - 9:30 AM | **Buffer Time**                               | Take a break to relax, grab a snack, or do a quick meditation to recharge.                     |
| 9:30 AM - 10:30 AM | **Electromagnetism Study Session**            | Dive into electromagnetism principles. Try to relate concepts to real-world applications, such as tech innovations. |
| 10:30 AM - 11:00 AM | **Buffer Time**                             | Stretch, walk around, or play a quick game of chess to stimulate your mind.                     |
| 11:00 AM - 12:00 PM | **Thermodynamics Study Session**              | Explore thermodynamics. Consider how these principles can be applied in energy-efficient startups. |
| 12:00 PM - 1:00 PM | **Lunch Break**                             | Enjoy a healthy meal and take time to relax.                                                   |
| 1:00 PM - 2:00 PM | **Entrepreneurship Workshop/Project**        | Participate in a workshop or work on a startup idea. Focus on how to apply scientific principles to business. |
| 2:00 PM - 2:30 PM | **Buffer Time**                             | Reflect on the workshop, jot down ideas, or engage in light reading related to entrepreneurship. |
| 2:30 PM - 3:30 PM | **Game-Based Learning (CounterStrike)**       | Play CounterStrike to enhance strategic thinking and teamwork. Consider how these skills apply to business. |
| 3:30 PM - 4:00 PM | **Buffer Time**                             | Wind down, engage in a casual conversation with friends, or play a few chess games.             |
| 4:00 PM - 5:00 PM | **Review and Reflect**                        | Review the day’s learning, consolidate knowledge, and plan for the next day.                    |
| 5:00 PM - 6:00 PM | **Extracurricular Activity (Chess)**          | Join a chess club or play online. Use this time to develop strategic thinking and patience.      |
| 6:00 PM - 7:00 PM | **Dinner Break**                             | Enjoy dinner and relax before the evening activities.                                          |
| 7:00 PM - 8:00 PM | **Networking/Startup Ideation**               | Connect with fellow students or mentors to discuss startup ideas and scientific contributions.    |
| 8:00 PM - 9:00 PM | **Leisure Time**                              | Engage in a hobby or relax with friends. Consider how leisure activities can inspire future projects. |
```